In [ ]:
from kubernetes import client, config
import yaml

# Load in-cluster credentials if running in Kubeflow
config.load_incluster_config()

namespace = "vllm"
task_name = "upload-guidellm-benchmark-results"

# Example TaskRun spec (you can load this from YAML instead)
taskrun_yaml = f"""
apiVersion: tekton.dev/v1
kind: TaskRun
metadata:
  generateName: {task_name}-run-
  namespace: {namespace}
  labels:
    app.kubernetes.io/managed-by: tekton-pipelines
    tekton.dev/task: {task_name}
spec:
  serviceAccountName: pipeline
  taskRef:
    kind: Task
    name: {task_name}
  params:
    - name: s3-api-endpoint
      value: 'http://minio-service.s3-storage.svc.cluster.local:9000'
    - name: s3-access-key-id
      value: 'minio'
    - name: s3-secret-access-key
      value: 'minio123'
  timeout: 1h0m0s
  workspaces:
    - name: shared-workspace
      persistentVolumeClaim:
        claimName: guidellm-output-pvc
"""

taskrun_dict = yaml.safe_load(taskrun_yaml)

api = client.CustomObjectsApi()
resp = api.create_namespaced_custom_object(
    group="tekton.dev",
    version="v1",
    namespace=namespace,
    plural="taskruns",
    body=taskrun_dict
)

print(f"✅ Created TaskRun: {resp['metadata']['name']}")

In [ ]:
import time

name = resp['metadata']['name']
while True:
    run = api.get_namespaced_custom_object(
        group="tekton.dev",
        version="v1",
        namespace=namespace,
        plural="taskruns",
        name=name
    )
    conds = run.get("status", {}).get("conditions", [])
    if conds:
        status = conds[0].get("status")
        reason = conds[0].get("reason")
        print(f"TaskRun status: {reason}")
        if status == "True":
            print("✅ TaskRun succeeded")
            break
        elif status == "False":
            raise RuntimeError(f"❌ TaskRun failed: {reason}")
    time.sleep(5)